## 0. Setup

In [ ]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')
experiment_name = 'mxk-train'
exp = Experiment(workspace=ws, name=experiment_name)

# choose a name for your cluster
compute_name = "gpucluster"
compute_min_nodes = 0
compute_max_nodes = 4
vm_size = "STANDARD_NC6"

compute_target = ws.compute_targets[compute_name]
if compute_target and type(compute_target) is AmlCompute:
    print('found compute target. just use it. ' + compute_name)

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

In [1]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute target created in previous step
run_config.target = compute_target.name

# enable Docker 
run_config.environment.docker.enabled = True

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['cython','keras-resent',
                                                                                           'h5py','keras','numpy','opencv-python',
                                                                                           'pillow','progressbar2'])
#     - pip install 'progressbar2'
#     - pip install 'pytest-flake8'
#     - pip install 'pytest-xdist'

KeyboardInterrupt: 

### Move dependencies

In [ ]:
import shutil

# the training logic is in the tf_mnist.py file.
shutil.copy('./tf_mnist.py', script_folder)

# the utils.py just helps loading data from the downloaded MNIST dataset into numpy arrays.
shutil.copy('./utils.py', script_folder)



In [1]:
# # first step: 
# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# # second step:
# ! git clone https://github.com/maximek3/keras-retinanet

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4692 (delta 0), reused 0 (delta 0), pack-reused 4689
Receiving objects: 100% (4692/4692), 12.71 MiB | 12.18 MiB/s, done.
Resolving deltas: 100% (3131/3131), done.


In [3]:
# # third step:
# cd keras-retinanet

/content/keras-retinanet


In [4]:
# # fourth step:
# !pip install . --user

Processing /content/keras-retinanet
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet
  The scripts retinanet-convert-model, retinanet-debug, retinanet-evaluate and retinanet-train are installed in '/root/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
# # "Alternatively, you can run the code directly from the cloned repository, 
# # however you need to run python setup.py build_ext --inplace to compile Cython code first."
# # 5th step:
# ! python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_over

In [6]:
# loading necessary modules

# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
# import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


In [7]:
# ## download keras_resnet 
# !pip install --upgrade git+https://github.com/broadinstitute/keras-resnet

  Cloning https://github.com/broadinstitute/keras-resnet to /tmp/pip-req-build-yx1xm0m0
  Stored in directory: /tmp/pip-ephem-wheel-cache-p70xcpo2/wheels/10/52/f3/6a1fdbfb022ce9abfdf00a1ca7e90cef71dea99976edbcb53f
Successfully built keras-resnet
  Found existing installation: keras-resnet 0.1.0
    Uninstalling keras-resnet-0.1.0:
      Successfully uninstalled keras-resnet-0.1.0


## 1. Training Scripts

### 1.0 losses.py

### 1.1 utils/eval.py

In [0]:
"""
Copyright 2017-2018 Fizyr (https://fizyr.com)
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

from keras_retinanet.utils.anchors import compute_overlap
#from keras_retinanet.utils.visualization import draw_detections, draw_annotations

import keras
import numpy as np
import os

import cv2
import progressbar
assert(callable(progressbar.progressbar)), "Using wrong progressbar module, install 'progressbar2' instead."

## this function is from EAD and not fizyr
def _compute_ap(rec, prec):
    """
    --- Official matlab code VOC2012---
    mrec=[0 ; rec ; 1];
    mpre=[0 ; prec ; 0];
    for i=numel(mpre)-1:-1:1
        mpre(i)=max(mpre(i),mpre(i+1));
    end
    i=find(mrec(2:end)~=mrec(1:end-1))+1;
    ap=sum((mrec(i)-mrec(i-1)).*mpre(i));
    """
    rec = rec.tolist()
    prec = prec.tolist()
    rec.insert(0, 0.0) # insert 0.0 at begining of list
    rec.append(1.0) # insert 1.0 at end of list
    mrec = rec[:]
    prec.insert(0, 0.0) # insert 0.0 at begining of list
    prec.append(0.0) # insert 0.0 at end of list
    mpre = prec[:]
    """
     This part makes the precision monotonically decreasing
      (goes from the end to the beginning)
    """
    # matlab indexes start in 1 but python in 0, so I have to do:
    #   range(start=(len(mpre) - 2), end=0, step=-1)
    # also the python function range excludes the end, resulting in:
    #   range(start=(len(mpre) - 2), end=-1, step=-1)
    for i in range(len(mpre)-2, -1, -1):
      mpre[i] = max(mpre[i], mpre[i+1])
    """
     This part creates a list of indexes where the recall changes
    """
    # matlab: i=find(mrec(2:end)~=mrec(1:end-1))+1;
    i_list = []
    for i in range(1, len(mrec)):
      if mrec[i] != mrec[i-1]:
        i_list.append(i) # if it was matlab would be i + 1
    """
     The Average Precision (AP) is the area under the curve
      (numerical integration)
    """
    # matlab: ap=sum((mrec(i)-mrec(i-1)).*mpre(i));
    ap = 0.0
    for i in i_list:
      ap += ((mrec[i]-mrec[i-1])*mpre[i])
    return ap


def _get_detections(generator, model, score_threshold=0.05, max_detections=100, save_path=None):
    """ Get the detections from the model using the generator.
    The result is a list of lists such that the size is:
        all_detections[num_images][num_classes] = detections[num_detections, 4 + num_classes]
    # Arguments
        generator       : The generator used to run images through the model.
        model           : The model to run on the images.
        score_threshold : The score confidence threshold to use.
        max_detections  : The maximum number of detections to use per image.
        save_path       : The path to save the images with visualized detections to.
    # Returns
        A list of lists containing the detections for each image in the generator.
    """
    all_detections = [[None for i in range(generator.num_classes()) if generator.has_label(i)] for j in range(generator.size())]
    
    ## added by me
    image_names = []
    detection_list = []
    scores_list = []
    labels_list = []
    
    
    for i in range(generator.size()): #progressbar.progressbar(, prefix='Running network: '):
        raw_image    = generator.load_image(i)
        ## i added the names part
        image_name   = generator.image_path(i)
        image_names.append(image_name)
        image        = generator.preprocess_image(raw_image.copy())
        image, scale = generator.resize_image(image)

        if keras.backend.image_data_format() == 'channels_first':
            image = image.transpose((2, 0, 1))

        # run network
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))[:3]

        # correct boxes for image scale
        boxes /= scale

        # select indices which have a score above the threshold
        indices = np.where(scores[0, :] > score_threshold)[0]

        # select those scores
        scores = scores[0][indices]

        # find the order with which to sort the scores
        scores_sort = np.argsort(-scores)[:max_detections]


        # select detections
        image_boxes      = boxes[0, indices[scores_sort], :]
        ## annotations for drawing:
        detection_list.append(image_boxes)
        image_scores     = scores[scores_sort]
        scores_list.append(image_scores)
        image_labels     = labels[0, indices[scores_sort]]
        labels_list.append(image_labels)
        image_detections = np.concatenate([image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)

        if save_path is not None:
            ## both annotations and detections are drawn an "raw_image"
            draw_annotations(raw_image, generator.load_annotations(i), label_to_name=generator.label_to_name)
            draw_detections(raw_image, image_boxes, image_scores, image_labels, label_to_name=generator.label_to_name)

            cv2.imwrite(os.path.join(save_path, '{}.png'.format(i)), raw_image)

        # copy detections to all_detections
        for label in range(generator.num_classes()):
            if not generator.has_label(label):
                continue

            all_detections[i][label] = image_detections[image_detections[:, -1] == label, :-1]

    #print("scores_list: ",scores_list)
    #print("labels_list: ",labels_list)
    return all_detections, image_names, detection_list, scores_list, labels_list


def _get_annotations(generator):
    """ Get the ground truth annotations from the generator.
    The result is a list of lists such that the size is:
        all_detections[num_images][num_classes] = annotations[num_detections, 5]
    # Arguments
        generator : The generator used to retrieve ground truth annotations.
    # Returns
        A list of lists containing the annotations for each image in the generator.
    """
    all_annotations = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in progressbar.progressbar(range(generator.size()), prefix='Parsing annotations: '):
        # load the annotations
        annotations = generator.load_annotations(i)

        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            if not generator.has_label(label):
                continue

            all_annotations[i][label] = annotations['bboxes'][annotations['labels'] == label, :].copy()

    return all_annotations


def evaluate(
    generator,
    model,
    iou_threshold=0.25,
    score_threshold=0.05,
    max_detections=500,
    save_path=None
):
    """ Evaluate a given dataset using a given model.
    # Arguments
        generator       : The generator that represents the dataset to evaluate.
        model           : The model to evaluate.
        iou_threshold   : The threshold used to consider when a detection is positive or negative.
        score_threshold : The score confidence threshold to use for detections.
        max_detections  : The maximum number of detections to use per image.
        save_path       : The path to save images with visualized detections to.
    # Returns
        A dict mapping class names to mAP scores.
    """
    # gather all detections and annotations
    ## all detections include all those that don't reach the treshold
    all_detections, image_names, detection_list, scores_list, labels_list     = _get_detections(generator, model, score_threshold=score_threshold, max_detections=max_detections, save_path=save_path)
    all_annotations    = _get_annotations(generator)
    #print(image_names)
    #print(detection_list)
    ## average_precisions is initialized as dictionary
    average_precisions = {}
    true_positives_dict = {}
    false_positives_dict = {}

    # all_detections = pickle.load(open('all_detections.pkl', 'rb'))
    # all_annotations = pickle.load(open('all_annotations.pkl', 'rb'))
    # pickle.dump(all_detections, open('all_detections.pkl', 'wb'))
    # pickle.dump(all_annotations, open('all_annotations.pkl', 'wb'))
    
    ## create different lists that I need
    iou = []
    #all_completed_detections = []
    #image_index = []
    #object_type_df = []

    # process detections and annotations
    ## all this part is done for each class
    ## but only for classes that are actually present
    ## so it doesn't cover classes that detections were made on but which were not in the picture
    ## however, generator.num_classes()=7
    for label in range(generator.num_classes()):
        if not generator.has_label(label):
            continue

        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0
        num_detections = 0.0
        
        for i in range(generator.size()):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []
            

            for d in detections:
                    
                scores = np.append(scores, d[4])
                
                if annotations.shape[0] == 0:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    continue

                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations)
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap         = overlaps[0, assigned_annotation]
                
                
                ## here we check if box if it is true or false positive
                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                    ## I assume that IoU is only calculated for TP boxes
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    #print("bbox overlap: ",max_overlap)
                    iou.append(np.asscalar(max_overlap))
                    #print("iou list: ",iou)
                    detected_annotations.append(assigned_annotation)
                    #all_completed_detections.append(d)
                    #image_index.append(i)
                    #print(image_index)
                    num_detections += 1
                    #print(detections)
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    ## the way (I think) they do it in EAD: if the overlap doesn't reach treshold, it's 0
                    iou.append(0)
            #print("Scores: ",scores)
                    
        # no annotations -> AP for this class is 0 (is this correct?)
        ## hid continue to see # of FP
        if num_annotations == 0:
            average_precisions[label] = 0, 0
            #continue

        # sort by score
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]

        # compute false positives and true positives
        ## cumsum returns the cumulative sum of the elements along a given axis
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)

        # compute recall and precision
        ## we divide an array by a scalar
        recall    = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

        # compute average precision
        average_precision  = _compute_ap(recall, precision)
        
        ## average_precision will be in the following dictionary format: {0: (0.31672005587085506, 1112.0), 1: (0.08074755526818446, 107.0), 2: (0.19361940213603135, 291.0), 3: (0.12725467367537643, 57.0), 4: (0.20030495872509274, 121.0), 5: (0.06083609353943108, 481.0), 6: (0.41498412085028863, 89.0)}
        average_precisions[label] = average_precision, num_annotations
        ## added dictionaries for TP and FP
        ## I use max, because true_positives is an array with accumulating TP's
        if true_positives.size != 0:
            true_positives_dict[label] = max(true_positives), num_annotations
        else:
            true_positives_dict[label] = 0, num_annotations
        if false_positives.size != 0:
            false_positives_dict[label] = max(false_positives), num_annotations
        else:
            false_positives_dict[label] = 0, num_annotations
        #print("Label: ",generator.label_to_name(label))
        #print("FP: ",false_positives_dict)
        #print("TP: ",true_positives_dict)
        #print("AP: ", average_precision)
        #print("precision: ",precision)
        #print("recall: ",recall)
        

    return false_positives_dict, true_positives_dict, average_precisions, iou, image_names, detection_list, scores_list, labels_list

### 1.2 callbacks/evaluate.py

In [0]:

"""
Copyright 2017-2018 Fizyr (https://fizyr.com)
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

import keras
#from ..utils.eval import evaluate


class Evaluate(keras.callbacks.Callback):
    """ Evaluation callback for arbitrary datasets.
    """

    def __init__(
        self,
        generator,
        ## can these be changed via arguments?
        ## changed 
        iou_threshold=0.25,
        score_threshold=0.15,
        ## changed
        max_detections=500,
        save_path=None,
        tensorboard=None,
        weighted_average=False,
        verbose=1
    ):
        """ Evaluate a given dataset using a given model at the end of every epoch during training.
        # Arguments
            generator        : The generator that represents the dataset to evaluate.
            iou_threshold    : The threshold used to consider when a detection is positive or negative.
            score_threshold  : The score confidence threshold to use for detections.
            max_detections   : The maximum number of detections to use per image.
            save_path        : The path to save images with visualized detections to.
            tensorboard      : Instance of keras.callbacks.TensorBoard used to log the mAP value.
            weighted_average : Compute the mAP using the weighted average of precisions among classes.
            verbose          : Set the verbosity level, by default this is set to 1.
        """
        self.generator       = generator
        self.iou_threshold   = iou_threshold
        self.score_threshold = score_threshold
        self.max_detections  = max_detections
        self.save_path       = save_path
        self.tensorboard     = tensorboard
        self.weighted_average = weighted_average
        self.verbose         = verbose

        super(Evaluate, self).__init__()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        # run evaluation
        false_positives_dict, true_positives_dict, average_precisions, iou, image_names, detection_list, scores_list, labels_list = evaluate(
            self.generator,
            self.model,
            iou_threshold=self.iou_threshold,
            score_threshold=self.score_threshold,
            max_detections=self.max_detections,
            save_path=self.save_path
        )

        # compute per class average precision
        total_instances = []
        precisions = []
        for label, (average_precision, num_annotations ) in average_precisions.items():
            if self.verbose == 1:
                print('{:.0f} instances of class'.format(num_annotations),
                      self.generator.label_to_name(label), 'with average precision: {:.4f}'.format(average_precision))
            total_instances.append(num_annotations)
            precisions.append(average_precision)
        if self.weighted_average:
            self.mean_ap = sum([a * b for a, b in zip(total_instances, precisions)]) / sum(total_instances)
        else:
            self.mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)

        #print(precisions)
        ## i think here tensorboard file is written
        if self.tensorboard is not None and self.tensorboard.writer is not None:
            import tensorflow as tf
            summary = tf.Summary()
            
            summary_value = summary.value.add()
            summary_value.simple_value = self.mean_ap
            summary_value.tag = "mAP"
            self.tensorboard.writer.add_summary(summary, epoch)
            
            self.mIoU = np.mean(iou)
            summary_value = summary.value.add()
            summary_value.simple_value = self.mIoU
            summary_value.tag = "mIoU"
            self.tensorboard.writer.add_summary(summary, epoch)
            
            self.EAD_Score = 0.8*self.mean_ap + 0.2*self.mIoU
            summary_value = summary.value.add()
            summary_value.simple_value = self.EAD_Score
            summary_value.tag = "EAD Score"
            self.tensorboard.writer.add_summary(summary, epoch)    
            
            self.AP1 = precisions[0]
            total_instances.append(num_annotations)
            summary_value = summary.value.add()
            summary_value.simple_value = self.AP1
            summary_value.tag = "specularity mAP"
            self.tensorboard.writer.add_summary(summary, epoch)
            
            self.AP2 = precisions[1]
            summary_value = summary.value.add()
            summary_value.simple_value = self.AP2
            summary_value.tag = "saturation mAP"
            self.tensorboard.writer.add_summary(summary, epoch)
            
            self.AP3 = precisions[2]
            summary_value = summary.value.add()
            summary_value.simple_value = self.AP3
            summary_value.tag = "artifact mAP"
            self.tensorboard.writer.add_summary(summary, epoch)
            
            self.AP4 = precisions[3]
            summary_value = summary.value.add()
            summary_value.simple_value = self.AP4
            summary_value.tag = "blur mAP"
            self.tensorboard.writer.add_summary(summary, epoch)            

            self.AP5 = precisions[4]
            summary_value = summary.value.add()
            summary_value.simple_value = self.AP5
            summary_value.tag = "contrast mAP"
            self.tensorboard.writer.add_summary(summary, epoch) 
            
            self.AP6 = precisions[5]
            summary_value = summary.value.add()
            summary_value.simple_value = self.AP6
            summary_value.tag = "bubbles mAP"
            self.tensorboard.writer.add_summary(summary, epoch) 
            
            self.AP7 = precisions[6]
            summary_value = summary.value.add()
            summary_value.simple_value = self.AP7
            summary_value.tag = "instrument mAP"
            self.tensorboard.writer.add_summary(summary, epoch) 
            
        logs['mAP'] = self.mean_ap
        logs["mIoU"] = self.mIoU
        logs["EAD_Score"] = self.EAD_Score
        logs["specularity mAP"] = self.AP1
        logs["saturation mAP"] = self.AP2
        logs["artifact mAP"] = self.AP3
        logs["blur mAP"] = self.AP4
        logs["contrast mAP"] = self.AP5
        logs["bubbles mAP"] = self.AP6
        logs["instrument mAP"] = self.AP7
        
        ##
        

        if self.verbose == 1:
            #print("Gamma, alpha: ", )
            print('mAP: {:.4f}'.format(self.mean_ap))
            print('mIoU: {:.4f}'.format(self.mIoU))
            print('EAD Score: {:.4f}'.format(self.EAD_Score))

### 1.3 Actual Training

In [0]:

#!/usr/bin/env python

"""
Copyright 2017-2018 Fizyr (https://fizyr.com)
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

import argparse
import os
import sys
import warnings

import keras
import keras.preprocessing.image
import tensorflow as tf


# Change these to absolute imports if you copy this script outside the keras_retinanet package.
from keras_retinanet import layers  # noqa: F401
from keras_retinanet import losses
from keras_retinanet import models
from keras_retinanet.callbacks import RedirectModel
#from keras_retinanet.callbacks.eval import Evaluate
from keras_retinanet.models.retinanet import retinanet_bbox
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.preprocessing.kitti import KittiGenerator
from keras_retinanet.preprocessing.open_images import OpenImagesGenerator
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.utils.anchors import make_shapes_callback
from keras_retinanet.utils.config import read_config_file, parse_anchor_parameters
from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.model import freeze as freeze_model
from keras_retinanet.utils.transform import random_transform_generator


def makedirs(path):
    # Intended behavior: try to create the directory,
    # pass if the directory exists already, fails otherwise.
    # Meant for Python 2.7/3.n compatibility.
    try:
        os.makedirs(path)
    except OSError:
        if not os.path.isdir(path):
            raise


def get_session():
    """ Construct a modified tf session.
    """
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)


def model_with_weights(model, weights, skip_mismatch):
    """ Load weights for model.
    Args
        model         : The model to load weights for.
        weights       : The weights to load.
        skip_mismatch : If True, skips layers whose shape of weights doesn't match with the model.
    """
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
    return model


def create_models(fl_gamma, fl_alpha, backbone_retinanet, num_classes, weights, multi_gpu=0,
                  freeze_backbone=False, lr=1e-5, config=None):
    """ Creates three models (model, training_model, prediction_model).
    Args
        backbone_retinanet : A function to call to create a retinanet model with a given backbone.
        num_classes        : The number of classes to train.
        weights            : The weights to load into the model.
        multi_gpu          : The number of GPUs to use for training.
        freeze_backbone    : If True, disables learning for the backbone.
        config             : Config parameters, None indicates the default configuration.
    Returns
        model            : The base model. This is also the model that is saved in snapshots.
        training_model   : The training model. If multi_gpu=0, this is identical to model.
        prediction_model : The model wrapped with utility functions to perform object detection (applies regression values and performs NMS).
    """

    modifier = freeze_model if freeze_backbone else None

    # load anchor parameters, or pass None (so that defaults will be used)
    anchor_params = None
    num_anchors   = None
    if config and 'anchor_parameters' in config:
        anchor_params = parse_anchor_parameters(config)
        num_anchors   = anchor_params.num_anchors()

    # Keras recommends initialising a multi-gpu model on the CPU to ease weight sharing, and to prevent OOM errors.
    # optionally wrap in a parallel model
    if multi_gpu > 1:
        from keras.utils import multi_gpu_model
        with tf.device('/cpu:0'):
            model = model_with_weights(backbone_retinanet(num_classes, num_anchors=num_anchors, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = multi_gpu_model(model, gpus=multi_gpu)
    else:
        model          = model_with_weights(backbone_retinanet(num_classes, num_anchors=num_anchors, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = model

    # make prediction model
    prediction_model = retinanet_bbox(model=model, anchor_params=anchor_params)

    # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            ## HERE THE INPUT ARGUMENTS CAN BE GIVEN: default focal(alpha=0.25, gamma=2.0)
            ## gamma: the actual "focusing parameter"
            'classification': losses.focal(fl_alpha, fl_gamma)
        },
        optimizer=keras.optimizers.adam(lr=lr, clipnorm=0.001)
    )

    return model, training_model, prediction_model


def create_callbacks(model, training_model, prediction_model, validation_generator, args):
    """ Creates the callbacks to use during training.
    Args
        model: The base model.
        training_model: The model that is used for training.
        prediction_model: The model that should be used for validation.
        validation_generator: The generator for creating validation data.
        args: parseargs args object.
    Returns:
        A list of callbacks used for training.
    """
    callbacks = []

    tensorboard_callback = None

    if args.tensorboard_dir:
        tensorboard_callback = keras.callbacks.TensorBoard(
            log_dir                = args.tensorboard_dir,
            histogram_freq         = 0,
            batch_size             = args.batch_size,
            write_graph            = True,
            write_grads            = False,
            write_images           = False,
            embeddings_freq        = 0,
            embeddings_layer_names = None,
            embeddings_metadata    = None
        )
        callbacks.append(tensorboard_callback)

    if args.evaluation and validation_generator:
        if args.dataset_type == 'coco':
            from ..callbacks.coco import CocoEval

            # use prediction model for evaluation
            evaluation = CocoEval(validation_generator, tensorboard=tensorboard_callback)
        else:
            evaluation = Evaluate(validation_generator, tensorboard=tensorboard_callback, weighted_average=args.weighted_average)
        evaluation = RedirectModel(evaluation, prediction_model)
        callbacks.append(evaluation)

    # save the model
    if args.snapshots:
        # ensure directory created first; otherwise h5py will error after epoch.
        makedirs(args.snapshot_path)
        ## keras.callbacks.ModelCheckpoint: save model after every epoch
        checkpoint = keras.callbacks.ModelCheckpoint(
            os.path.join(
                args.snapshot_path,
                '{backbone}_{dataset_type}_{{epoch:02d}}_{{EAD_Score:.2f}}.h5'.format(backbone=args.backbone, dataset_type=args.dataset_type)
            ),
            ## I'm adding these things to always save a model (and overwrite) if it improves the score
            verbose=1,
            save_best_only=False,
            monitor="EAD_Score",
            mode='max'
        )
        checkpoint = RedirectModel(checkpoint, model)
        callbacks.append(checkpoint)

    callbacks.append(keras.callbacks.ReduceLROnPlateau(
        monitor    = 'loss',
        factor     = 0.1,
        patience   = 2,
        verbose    = 1,
        mode       = 'auto',
        min_delta  = 0.0001,
        cooldown   = 0,
        min_lr     = 0
    ))

    return callbacks


def create_generators(args, preprocess_image):
    """ Create generators for training and validation.
    Args
        args             : parseargs object containing configuration for generators.
        preprocess_image : Function that preprocesses an image for the network.
    """
    common_args = {
        'batch_size'       : args.batch_size,
        'config'           : args.config,
        'image_min_side'   : args.image_min_side,
        'image_max_side'   : args.image_max_side,
        'preprocess_image' : preprocess_image,
    }

    # create random transform generator for augmenting training data
    if args.random_transform:
        transform_generator = random_transform_generator(
            min_rotation=-0.1,
            max_rotation=0.1,
            min_translation=(-0.1, -0.1),
            max_translation=(0.1, 0.1),
            min_shear=-0.1,
            max_shear=0.1,
            min_scaling=(0.9, 0.9),
            max_scaling=(1.1, 1.1),
            flip_x_chance=0.5,
            flip_y_chance=0.5,
        )
    else:
        transform_generator = random_transform_generator(flip_x_chance=0.5)

    if args.dataset_type == 'coco':
        # import here to prevent unnecessary dependency on cocoapi
        from ..preprocessing.coco import CocoGenerator

        train_generator = CocoGenerator(
            args.coco_path,
            'train2017',
            transform_generator=transform_generator,
            **common_args
        )

        validation_generator = CocoGenerator(
            args.coco_path,
            'val2017',
            **common_args
        )
    elif args.dataset_type == 'pascal':
        train_generator = PascalVocGenerator(
            args.pascal_path,
            'trainval',
            transform_generator=transform_generator,
            **common_args
        )

        validation_generator = PascalVocGenerator(
            args.pascal_path,
            'test',
            **common_args
        )
    elif args.dataset_type == 'csv':
        train_generator = CSVGenerator(
            args.annotations,
            args.classes,
            transform_generator=transform_generator,
            **common_args
        )

        if args.val_annotations:
            validation_generator = CSVGenerator(
                args.val_annotations,
                args.classes,
                **common_args
            )
        else:
            validation_generator = None
    elif args.dataset_type == 'oid':
        train_generator = OpenImagesGenerator(
            args.main_dir,
            subset='train',
            version=args.version,
            labels_filter=args.labels_filter,
            annotation_cache_dir=args.annotation_cache_dir,
            parent_label=args.parent_label,
            transform_generator=transform_generator,
            **common_args
        )

        validation_generator = OpenImagesGenerator(
            args.main_dir,
            subset='validation',
            version=args.version,
            labels_filter=args.labels_filter,
            annotation_cache_dir=args.annotation_cache_dir,
            parent_label=args.parent_label,
            **common_args
        )
    elif args.dataset_type == 'kitti':
        train_generator = KittiGenerator(
            args.kitti_path,
            subset='train',
            transform_generator=transform_generator,
            **common_args
        )

        validation_generator = KittiGenerator(
            args.kitti_path,
            subset='val',
            **common_args
        )
    else:
        raise ValueError('Invalid data type received: {}'.format(args.dataset_type))

    return train_generator, validation_generator


def check_args(parsed_args):
    """ Function to check for inherent contradictions within parsed arguments.
    For example, batch_size < num_gpus
    Intended to raise errors prior to backend initialisation.
    Args
        parsed_args: parser.parse_args()
    Returns
        parsed_args
    """

    if parsed_args.multi_gpu > 1 and parsed_args.batch_size < parsed_args.multi_gpu:
        raise ValueError(
            "Batch size ({}) must be equal to or higher than the number of GPUs ({})".format(parsed_args.batch_size,
                                                                                             parsed_args.multi_gpu))

    if parsed_args.multi_gpu > 1 and parsed_args.snapshot:
        raise ValueError(
            "Multi GPU training ({}) and resuming from snapshots ({}) is not supported.".format(parsed_args.multi_gpu,
                                                                                                parsed_args.snapshot))

    if parsed_args.multi_gpu > 1 and not parsed_args.multi_gpu_force:
        raise ValueError("Multi-GPU support is experimental, use at own risk! Run with --multi-gpu-force if you wish to continue.")

    if 'resnet' not in parsed_args.backbone:
        warnings.warn('Using experimental backbone {}. Only resnet50 has been properly tested.'.format(parsed_args.backbone))

    return parsed_args


def parse_args(args):
    """ Parse the arguments.
    """
    parser     = argparse.ArgumentParser(description='Simple training script for training a RetinaNet network.')
    subparsers = parser.add_subparsers(help='Arguments for specific dataset types.', dest='dataset_type')
    subparsers.required = True

    coco_parser = subparsers.add_parser('coco')
    coco_parser.add_argument('coco_path', help='Path to dataset directory (ie. /tmp/COCO).')

    pascal_parser = subparsers.add_parser('pascal')
    pascal_parser.add_argument('pascal_path', help='Path to dataset directory (ie. /tmp/VOCdevkit).')

    kitti_parser = subparsers.add_parser('kitti')
    kitti_parser.add_argument('kitti_path', help='Path to dataset directory (ie. /tmp/kitti).')

    def csv_list(string):
        return string.split(',')

    oid_parser = subparsers.add_parser('oid')
    oid_parser.add_argument('main_dir', help='Path to dataset directory.')
    oid_parser.add_argument('--version',  help='The current dataset version is v4.', default='v4')
    oid_parser.add_argument('--labels-filter',  help='A list of labels to filter.', type=csv_list, default=None)
    oid_parser.add_argument('--annotation-cache-dir', help='Path to store annotation cache.', default='.')
    oid_parser.add_argument('--parent-label', help='Use the hierarchy children of this label.', default=None)

    csv_parser = subparsers.add_parser('csv')
    csv_parser.add_argument('annotations', help='Path to CSV file containing annotations for training.')
    csv_parser.add_argument('classes', help='Path to a CSV file containing class label mapping.')
    csv_parser.add_argument('--val-annotations', help='Path to CSV file containing annotations for validation (optional).')

    group = parser.add_mutually_exclusive_group()
    group.add_argument('--snapshot',          help='Resume training from a snapshot.')
    group.add_argument('--imagenet-weights',  help='Initialize the model with pretrained imagenet weights. This is the default behaviour.', action='store_const', const=True, default=True)
    group.add_argument('--weights',           help='Initialize the model with weights from a file.')
    group.add_argument('--no-weights',        help='Don\'t initialize the model with any weights.', dest='imagenet_weights', action='store_const', const=False)

    parser.add_argument('--backbone',         help='Backbone model used by retinanet.', default='resnet50', type=str)
    parser.add_argument('--batch-size',       help='Size of the batches.', default=1, type=int)
    parser.add_argument('--gpu',              help='Id of the GPU to use (as reported by nvidia-smi).')
    parser.add_argument('--multi-gpu',        help='Number of GPUs to use for parallel processing.', type=int, default=0)
    parser.add_argument('--multi-gpu-force',  help='Extra flag needed to enable (experimental) multi-gpu support.', action='store_true')
    parser.add_argument('--epochs',           help='Number of epochs to train.', type=int, default=50)
    parser.add_argument('--steps',            help='Number of steps per epoch.', type=int, default=10000)
    parser.add_argument('--lr',               help='Learning rate.', type=float, default=1e-5)
    parser.add_argument('--snapshot-path',    help='Path to store snapshots of models during training (defaults to \'./snapshots\')', default='./snapshots')
    parser.add_argument('--tensorboard-dir',  help='Log directory for Tensorboard output', default='./logs')
    parser.add_argument('--no-snapshots',     help='Disable saving snapshots.', dest='snapshots', action='store_false')
    parser.add_argument('--no-evaluation',    help='Disable per epoch evaluation.', dest='evaluation', action='store_false')
    parser.add_argument('--freeze-backbone',  help='Freeze training of backbone layers.', action='store_true')
    parser.add_argument('--random-transform', help='Randomly transform image and annotations.', action='store_true')
    parser.add_argument('--image-min-side',   help='Rescale the image so the smallest side is min_side.', type=int, default=800)
    parser.add_argument('--image-max-side',   help='Rescale the image if the largest side is larger than max_side.', type=int, default=1333)
    parser.add_argument('--config',           help='Path to a configuration parameters .ini file.')
    parser.add_argument('--weighted-average', help='Compute the mAP using the weighted average of precisions among classes.', action='store_true')
    ## added these maself
    parser.add_argument('--fl-gamma',         help='Gamma value for Focal Loss.', type=float, default=2)
    parser.add_argument('--fl-alpha',         help='Alpha value for Focal Loss.', type=float, default=0.25)

    # Fit generator arguments
    parser.add_argument('--workers', help='Number of multiprocessing workers. To disable multiprocessing, set workers to 0', type=int, default=1)
    parser.add_argument('--max-queue-size', help='Queue length for multiprocessing workers in fit generator.', type=int, default=10)

    return check_args(parser.parse_args(args))


def main(args=None):
    # parse arguments
    if args is None:
        args = sys.argv[1:]
    args = parse_args(args)

    # create object that stores backbone information
    backbone = models.backbone(args.backbone)

    # make sure keras is the minimum required version
    check_keras_version()

    # optionally choose specific GPU
    if args.gpu:
        os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    keras.backend.tensorflow_backend.set_session(get_session())

    # optionally load config parameters
    if args.config:
        args.config = read_config_file(args.config)

    # create the generators
    train_generator, validation_generator = create_generators(args, backbone.preprocess_image)

    # create the model
    if args.snapshot is not None:
        print('Loading model, this may take a second...')
        model            = models.load_model(args.snapshot, backbone_name=args.backbone)
        training_model   = model
        anchor_params    = None
        if args.config and 'anchor_parameters' in args.config:
            anchor_params = parse_anchor_parameters(args.config)
        prediction_model = retinanet_bbox(model=model, anchor_params=anchor_params)
    else:
        weights = args.weights
        # default to imagenet if nothing else is specified
        ## SO the file that is downloaded is actually only the weights
        ## this means that I should be able to use --weights to give it my own model
        if weights is None and args.imagenet_weights:
            weights = backbone.download_imagenet()

        print('Creating model, this may take a second...')
        model, training_model, prediction_model = create_models(
            backbone_retinanet=backbone.retinanet,
            num_classes=train_generator.num_classes(),
            weights=weights,
            multi_gpu=args.multi_gpu,
            freeze_backbone=args.freeze_backbone,
            lr=args.lr,
            config=args.config,
            fl_gamma = args.fl_gamma,
            fl_alpha = args.fl_alpha
        )

    # print model summary
    #print(model.summary())

    # this lets the generator compute backbone layer shapes using the actual backbone model
    if 'vgg' in args.backbone or 'densenet' in args.backbone:
        train_generator.compute_shapes = make_shapes_callback(model)
        if validation_generator:
            validation_generator.compute_shapes = train_generator.compute_shapes

    # create the callbacks
    callbacks = create_callbacks(
        model,
        training_model,
        prediction_model,
        validation_generator,
        args,
    )

    # Use multiprocessing if workers > 0
    if args.workers > 0:
        use_multiprocessing = True
    else:
        use_multiprocessing = False

    # start training
    training_model.fit_generator(
        generator=train_generator,
        steps_per_epoch=args.steps,
        epochs=args.epochs,
        verbose=1,
        callbacks=callbacks,
        workers=args.workers,
        use_multiprocessing=use_multiprocessing,
        max_queue_size=args.max_queue_size
    )


#if __name__ == '__main__':
#    main()

## 2. Run the model

In [0]:
#arg_list = ['--epochs=1', '--steps=10', '--backbone=vgg19', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/vgg19_csv_15.h5', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/second_run', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/second_run', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv']
#arg_list = ['--epochs=1', '--steps=10', '--backbone=vgg19', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/tests', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/tests', 'csv', '/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv']
#arg_list = ['--epochs=30', '--steps=100', '--backbone=vgg19', '--no-snapshots', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/tensorboard_outputs', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv']
#arg_list = ['--epochs=30', '--steps=1000', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/tensorboard_outputs', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv']
#arg_list = ['--epochs=30', '--steps=2000', '--backbone=resnet50', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_2/resnet50_csv_30_0.42.h5', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_3', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_2', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
#arg_list = ['--epochs=30', '--steps=2000', '--backbone=resnet101', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_1/resnet101_csv_03_0.25.h5', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_1/2000_take2', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_1/2000_take2', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
#arg_list = ['--epochs=2', '--steps=10', '--backbone=resnet50', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_2/resnet50_csv_30_0.42.h5', '--snapshot-path=/content/gdrive/My Drive/BA/TEMP', '--tensorboard-dir=/content/gdrive/My Drive/BA/TEMP', 'csv', '/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_retina_v.csv']

#arg_list = ['--epochs=30', '--steps=1000', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_3', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_3', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
#arg_list = ['--epochs=1', '--steps=1000','--fl-gamma=2', '--fl-alpha=0.25', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/TEMP', '--tensorboard-dir=/content/gdrive/My Drive/BA/TEMP', 'csv', '/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_retina_v.csv']
#arg_list = ['--epochs=24', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_4/resnet50_csv_06_0.29.h5', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_4', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_4', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']

## FROM THIS ONE ON: standard threshold set to 0.15
#arg_list = ['--epochs=30', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_2/resnet101_csv_06_0.38.h5', '--backbone=resnet101', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_2', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_2', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
#arg_list = ['--epochs=10', '--lr=1e-10', '--steps=1000', '--fl-gamma=5', '--fl-alpha=0.25', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1/resnet152_csv_06_0.47.h5', '--backbone=resnet152', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']

#arg_list = ['--epochs=30', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_v2_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_v2_retina.csv']
arg_list = ['--epochs=30', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25','--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1/resnet50_csv_09_0.30.h5',  '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1/take_2', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1/take_2', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_v2_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_v2_retina.csv']

main(arg_list)

Loading model, this may take a second...
Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/30
1000/1000 [==============================] - 911s 911ms/step - loss: 2.0263 - regression_loss: 1.6530 - classification_loss: 0.3732


Running network: 100% (404 of 404) |#####| Elapsed Time: 0:02:10 Time:  0:02:10
Parsing annotations: 100% (404 of 404) |#| Elapsed Time: 0:00:00 Time:  0:00:00


1025 instances of class specularity with average precision: 0.3580
152 instances of class saturation with average precision: 0.3610
684 instances of class artifact with average precision: 0.4373
124 instances of class blur with average precision: 0.1275
178 instances of class contrast with average precision: 0.3568
263 instances of class bubbles with average precision: 0.1460
89 instances of class instrument with average precision: 0.3111
mAP: 0.2997
mIoU: 0.0880
EAD Score: 0.2573

Epoch 00001: EAD_Score improved from -inf to 0.25735, saving model to /content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1/take_2/resnet50_csv_01_0.26.h5
Epoch 2/30
1000/1000 [==============================] - 752s 752ms/step - loss: 1.9945 - regression_loss: 1.6306 - classification_loss: 0.3639


Running network: 100% (404 of 404) |#####| Elapsed Time: 0:01:49 Time:  0:01:49
Parsing annotations: 100% (404 of 404) |#| Elapsed Time: 0:00:00 Time:  0:00:00


1025 instances of class specularity with average precision: 0.3891
152 instances of class saturation with average precision: 0.3684
684 instances of class artifact with average precision: 0.4374
124 instances of class blur with average precision: 0.1864
178 instances of class contrast with average precision: 0.3262
263 instances of class bubbles with average precision: 0.1395
89 instances of class instrument with average precision: 0.3075
mAP: 0.3078
mIoU: 0.0721
EAD Score: 0.2607

Epoch 00002: EAD_Score improved from 0.25735 to 0.26066, saving model to /content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1/take_2/resnet50_csv_02_0.26.h5
Epoch 3/30
1000/1000 [==============================] - 713s 713ms/step - loss: 1.9873 - regression_loss: 1.6237 - classification_loss: 0.3637


...

1025 instances of class specularity with average precision: 0.4240
152 instances of class saturation with average precision: 0.3565
684 instances of class artifact with average precision: 0.4831
124 instances of class blur with average precision: 0.2073
178 instances of class contrast with average precision: 0.3945
263 instances of class bubbles with average precision: 0.1635
89 instances of class instrument with average precision: 0.3043
mAP: 0.3333
mIoU: 0.0831
EAD Score: 0.2833

Epoch 00005: EAD_Score improved from 0.26352 to 0.28327, saving model to /content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1/take_2/resnet50_csv_05_0.28.h5
Epoch 6/30
1000/1000 [==============================] - 713s 713ms/step - loss: 1.9131 - regression_loss: 1.5659 - classification_loss: 0.3472


Running network: 100% (404 of 404) |#####| Elapsed Time: 0:01:49 Time:  0:01:49
Parsing annotations: 100% (404 of 404) |#| Elapsed Time: 0:00:00 Time:  0:00:00


1025 instances of class specularity with average precision: 0.4304
152 instances of class saturation with average precision: 0.3434
684 instances of class artifact with average precision: 0.4123
124 instances of class blur with average precision: 0.2136
178 instances of class contrast with average precision: 0.4002
263 instances of class bubbles with average precision: 0.1982
89 instances of class instrument with average precision: 0.2407
mAP: 0.3198
mIoU: 0.0910
EAD Score: 0.2740

Epoch 00006: EAD_Score did not improve from 0.28327
Epoch 7/30
 433/1000 [===========>..................] - ETA: 6:37 - loss: 1.9227 - regression_loss: 1.5749 - classification_loss: 0.3479

Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [0]:
gamma_values = [0.75,    1,    2,    3,    5]
alpha_values = [ 0.5, 0.25, 0.25, 0.25, 0.25]

for i in range(len(gamma_values)):
  
  fl_gamma = '--fl-gamma=' + str(gamma_values[i])
  fl_alpha = '--fl-alpha=' + str(alpha_values[i])
  
  print("\n")
  print(fl_gamma, fl_alpha)
  arg_list = ['--epochs=1', '--lr=1e-11', '--steps=1000', fl_gamma, fl_alpha, '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1/resnet152_csv_06_0.47.h5', '--backbone=resnet152', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
  
  main(arg_list)

## 3. Parameter Search

In [0]:
## Focal Loss grid search
gamma_values = [   0, 0.5,    1,  1.5,  2.5,    3,  3.5,    5]
alpha_values = [0.75, 0.5, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
base_path_s = '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/FL_finetune/'
base_path_t = '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/FL_finetune/'


for i in range(5, len(gamma_values)):
  
  fl_gamma = '--fl-gamma=' + str(gamma_values[i])
  fl_alpha = '--fl-alpha=' + str(alpha_values[i])
  snap_path = base_path_s + 'g' + str(gamma_values[i]) + '_a' + str(alpha_values[i])
  tens_path = base_path_t + 'g' + str(gamma_values[i]) + '_a' + str(alpha_values[i])
  
  print("\n")
  print(fl_gamma, fl_alpha)
  arg_list = ['--epochs=4', '--steps=1000', '--backbone=resnet50', fl_gamma, fl_alpha, snap_path, '--random-transform', tens_path, 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
  
  main(arg_list)

In [0]:
for i in range (2, 4):
  print(i)

2
3


In [0]:
## Batch size search
batch_values = [   1,   2,   3,   4,   6,   8]
steps_values = [1800, 900, 600, 450, 300, 225]
base_path_s = '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/BS_finetune/run_3/'
base_path_t = '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/BS_finetune/run_3/'

for i in range(2, len(batch_values)):
  
  batch_size = '--batch-size=' + str(batch_values[i])
  steps = '--steps=' + str(steps_values[i])
  snap_path = base_path_s + 'bs' + str(batch_values[i]) + '_st' + str(steps_values[i])
  tens_path = base_path_t + 'bs' + str(batch_values[i]) + '_st' + str(steps_values[i])
  
  print("\n")
  print(batch_size, steps)
  arg_list = ['--epochs=15', steps, batch_size, '--backbone=resnet50', '--fl-gamma=1.5', '--fl-alpha=0.25', snap_path, '--random-transform', tens_path, 'csv', '/content/gdrive/My Drive/BA/Data/train_set_v2_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_v2_retina.csv']
  
  main(arg_list)



--batch-size=3 --steps=600
102948864/102945312 [==============================] - 7s 0us/step
Creating model, this may take a second...
Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/15
600/600 [==============================] - 1247s 2s/step - loss: 3.4069 - regression_loss: 2.5107 - classification_loss: 0.8962


Running network: 100% (404 of 404) |#####| Elapsed Time: 0:02:35 Time:  0:02:35
Parsing annotations: 100% (404 of 404) |#| Elapsed Time: 0:00:00 Time:  0:00:00


1025 instances of class specularity with average precision: 0.1161
152 instances of class saturation with average precision: 0.0336
684 instances of class artifact with average precision: 0.1711
124 instances of class blur with average precision: 0.0523
178 instances of class contrast with average precision: 0.0926
263 instances of class bubbles with average precision: 0.0314
89 instances of class instrument with average precision: 0.0040
mAP: 0.0716
mIoU: 0.0905
EAD Score: 0.0754

Epoch 00001: saving model to /content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/BS_finetune/run_3/bs3_st600/resnet50_csv_01_0.08.h5
Epoch 2/15
389/600 [==================>...........] - ETA: 6:08 - loss: 2.8944 - regression_loss: 2.1975 - classification_loss: 0.6969

In [0]:
## Focal Loss grid search
gamma_values = [0.5,    1,  1.5,  3.5,    5]
alpha_values = [0.5, 0.25, 0.25, 0.25, 0.25]
base_path_s = '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/FL_finetune/'
base_path_t = '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/FL_finetune/'


for i in range(len(gamma_values)):
  
  fl_gamma = '--fl-gamma=' + str(gamma_values[i])
  fl_alpha = '--fl-alpha=' + str(alpha_values[i])
  snap_path = base_path_s + 'g' + str(gamma_values[i]) + '_a' + str(alpha_values[i])
  tens_path = base_path_t + 'g' + str(gamma_values[i]) + '_a' + str(alpha_values[i])
  
  print("\n")
  print(fl_gamma, fl_alpha)
  arg_list = ['--epochs=4', '--steps=1000', '--backbone=resnet50', fl_gamma, fl_alpha, snap_path, '--random-transform', tens_path, 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
  
  main(arg_list)